In [ ]:
# IU data set curation into the same strcture as Mimic

# The IU dataset was downloaded from:
# https://openi.nlm.nih.gov/imgs/collections/NLMCXR_png.tgz
# https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz
# or use the API

# Parts of this notebook have been adapted from the following sources:
# https://www.kaggle.com/code/kundnjha/analysis-of-chest-x-rays-indiana-university/notebook

In [ ]:
!python -m pip install beautifulsoup4 lxml matplotlib opencv-python seaborn tqdm wordcloud

In [ ]:
# Load required packages
import os
import re
import shutil
import warnings
from collections import Counter

import cv2
import lxml
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm import tqdm

sns.set_style("whitegrid")
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")
import xml.etree.ElementTree as ET

from wordcloud import WordCloud

In [ ]:
REPORTS_LOC = "../data/xray_reports"
IMAGES_LOC = "../data/xray_images"

In [ ]:
# Check the number of images and reports from downloaded data

print("Total Images in data : ", len(os.listdir(IMAGES_LOC)))
print("Total Reports in data : ", len(os.listdir(REPORTS_LOC)))

In [ ]:
# List of count of images and reports pairs

img_count = []
for file in os.listdir(REPORTS_LOC):
    xml_file = os.path.join(REPORTS_LOC, file)
    # reading the xml data
    with open(xml_file, "r") as f:
        data = f.read()
    # getting all the image names
    regex = r"parentImage id.*"
    k = re.findall(regex, data)
    temp = len(k)
    img_count.append(temp)

print("The max number of images associated with a report:", np.array(img_count).max())
print("The min number of images associated with a report:", np.array(img_count).min())

In [ ]:
# Plot the above image counts

plt.figure(figsize=(6, 5))
ax = pd.Series(img_count).plot(kind="hist", color="blue")
ax.set_xlabel("Number of images associated with report")
ax.set_title("Frequency VS Number of images associated with report")
plt.show()

In [ ]:
# Print the number of exact image counts per patient

print("Images per patient :\n")
print(pd.Series(img_count).value_counts())

In [ ]:
# Extract headings from report xml files and save into a dataframe
# Reference : https://stackoverflow.com/questions/2723015/how-to-find-recursively-for-a-tag-of-xml-using-lxml

columns = [
    "image_name",
    "image_caption",
    "comparison",
    "indication",
    "findings",
    "impression",
    "mesh",
]
report_df = pd.DataFrame(columns=columns)
for file in tqdm(os.listdir(REPORTS_LOC)):
    # find files with .xml extension only
    if file.endswith(".xml"):
        # finding root element
        tree = ET.parse(REPORTS_LOC + "/" + file)  # parse the xml file

        findings = tree.find(".//AbstractText[@Label='FINDINGS']").text
        indication = tree.find(".//AbstractText[@Label='INDICATION']").text
        comparision = tree.find(".//AbstractText[@Label='COMPARISON']").text
        impression = tree.find(".//AbstractText[@Label='IMPRESSION']").text

        caption = set()
        name_img = set()
        # find images in each parentImage tag
        for iterator in tree.findall("parentImage"):
            img = iterator.attrib["id"] + ".png"
            name_img.add(img)
            # add the corresponding report for each image
            caption.add(
                ""
                if iterator.find("caption").text is None
                else iterator.find("caption").text
            )

        mesh = set()
        # find images in each parentImage tag
        for iterator in tree.findall("MeSH"):
            mesh.add(
                ""
                if iterator.find("major").text is None
                else iterator.find("major").text
            )

        # add image details and reports to dataframe
        report_df = report_df.append(
            pd.Series(
                [
                    ",".join(name_img),
                    ",".join(caption),
                    comparision,
                    indication,
                    findings,
                    impression,
                    ",".join(mesh),
                ],
                index=columns,
            ),
            ignore_index=True,
        )

In [ ]:
# Show the dataframe created in the above cell

report_df.head()

In [ ]:
# Add a column for the number of images per case

report_df["image_count"] = report_df["image_name"].astype(str).str.split(",").apply(len)
report_df.head()

In [ ]:
# Seperate each image onto a seperate line in new dataframe df1

report_df.image_name = report_df.image_name.str.split(",")
report_df_explode = report_df.explode("image_name")

In [ ]:
report_df_explode.head()

In [ ]:
# Add new column for the id of each case taken from the first letters of the image name

report_df_explode["id"] = report_df_explode["image_name"].str.extract(r"^([^_]+)")
report_df_explode.head()

In [ ]:
# Find NA values in reports

NaNs = report_df_explode.isnull().sum()
print("Total Nan Values in caption columns -", NaNs[1])
print("Total Nan Values in comparison columns -", NaNs[2])
print("Total Nan Values in Indication columns -", NaNs[3])
print("Total Nan Values in findings columns   -", NaNs[4])
print("Total Nan Values in Impression columns -", NaNs[5])

In [ ]:
# Replacing the nan values in reports if missing

report_df_explode["image_caption"] = report_df_explode["image_caption"].fillna(
    "Unknown"
)
report_df_explode["comparison"] = report_df_explode["comparison"].fillna(
    "No Comparison"
)
report_df_explode["indication"] = report_df_explode["indication"].fillna(
    "No Indication"
)
report_df_explode["findings"] = report_df_explode["findings"].fillna("No Findings")
report_df_explode["impression"] = report_df_explode["impression"].fillna(
    "No Impression"
)

In [ ]:
# Check the changes to NA values

NaNs_filled = report_df_explode.isnull().sum()
print("Total Nan Values in caption columns -", NaNs_filled[1])
print("Total Nan Values in comparison columns -", NaNs_filled[2])
print("Total Nan Values in Indication columns -", NaNs_filled[3])
print("Total Nan Values in findings columns   -", NaNs_filled[4])
print("Total Nan Values in Impression columns -", NaNs_filled[5])

In [ ]:
# Replace XX values to __ like in MIMIC


def preprocess_text(data):  # https://regex101.com/
    """
    extracts the information data from the xml file and does text preprocessing on them
    here info can be 1 value in this list ["COMPARISON","INDICATION","FINDINGS","IMPRESSION"]
    """
    preprocessed = []

    for sentence in tqdm(data.values):

        sentence = BeautifulSoup(sentence, "lxml").get_text()

        regex = r"XX+"
        sentence = re.sub(regex, "___", sentence)  # removing words like XXXX

        if (
            sentence == ""
        ):  # if the resulting sentence is an empty string return null value
            sentence = np.nan
        preprocessed.append(sentence)
    return preprocessed

In [ ]:
# Apply preprocessing to reports

report_df_explode["image_caption"] = preprocess_text(report_df_explode["image_caption"])
report_df_explode["comparison"] = preprocess_text(report_df_explode["comparison"])
report_df_explode["indication"] = preprocess_text(report_df_explode["indication"])
report_df_explode["findings"] = preprocess_text(report_df_explode["findings"])
report_df_explode["impression"] = preprocess_text(report_df_explode["impression"])

report_df_explode.head()

In [ ]:
# Find and remove empty cells with no images

report_df_explode.replace("", float("NaN"), inplace=True)
print(report_df_explode.isnull().sum() * 100 / report_df_explode.shape[0])
print()
report_df_explode.dropna(subset=["image_name"], inplace=True)
print(report_df_explode.isnull().sum() * 100 / report_df_explode.shape[0])
report_df_explode.shape

In [ ]:
# Adding word count feature for indication, findings and impression

report_df_explode["indication_count"] = (
    report_df_explode["indication"]
    .astype(str)
    .str.split()
    .apply(lambda x: 0 if x == None else len(x))
)
report_df_explode["findings_count"] = (
    report_df_explode["findings"]
    .astype(str)
    .str.split()
    .apply(lambda x: 0 if x == None else len(x))
)
report_df_explode["impression_count"] = (
    report_df_explode["impression"]
    .astype(str)
    .str.split()
    .apply(lambda x: 0 if x == None else len(x))
)
report_df_explode.head()

In [ ]:
report_df_explode["impression"] = (
    report_df_explode["impression"]
    .astype(str)
    .apply(lambda x: x + "." if x[-1] != "." else x)
)

In [ ]:
# combine impression and findings
report_df_explode["report"] = (
    report_df_explode["impression"] + " " + report_df_explode["findings"]
)
report_df_explode.head()

In [ ]:
reports_clean = report_df_explode[["id", "report"]]
reports_clean.head()

In [ ]:
# save output impresion and findings
reports_clean.to_csv("./reports_clean.csv", header=False, index=False)

In [ ]:
reports_impression = report_df_explode[["id", "impression"]]

In [ ]:
# save output impression only
reports_impression.to_csv("./reports_impression.csv", header=False, index=False)

In [ ]:
# These report files are then used the generate the CheXpert lables using the Stanford ML Group code
# https://stanfordmlgroup.github.io/competitions/chexpert/

In [ ]:
# Create folder structure for IU data

In [ ]:
# Add a new column with images labled as .jpg instead of .png

report_df_explode["image_name_jpg"] = report_df_explode["image_name"].str.replace(
    ".png", ".jpg"
)
report_df_explode.head()

In [ ]:
# Create a new dataframe with just the id and image name

image_id_name = report_df_explode[["id", "image_name_jpg"]]
image_id_name.head()

In [ ]:
# Convert png images to jpg
output_loc = r"../data/xray_images_jpg"
os.makedirs(output_loc, exist_ok=True)

files_to_convert = (f for f in os.listdir(IMAGES_LOC) if f.endswith(".png"))
for filename in files_to_convert:
    im = Image.open(os.path.join(IMAGES_LOC, filename))
    root, _ = os.path.splitext(filename)
    jpg_file = os.path.join(output_loc, f"{root}.jpg")
    im.save(jpg_file)

In [ ]:
# Copy images to seperate files to replicate mimic folder structure
# Note the folder structure is one layer less deep than mimic

src_dir = "../data/xray_images_jpg"
dst_dir = "../data/xray"

# Seperate images out into their respective id folders
for name, s in image_id_name.groupby("id")["image_name_jpg"]:
    full_dir = os.path.join(dst_dir, name)
    print(f"copying {s.shape[0]} images from {src_dir} to {full_dir}")
    os.makedirs(full_dir, exist_ok=True)
    for filename in s:
        source = os.path.join(src_dir, filename)
        shutil.copy(source, full_dir)

In [ ]:
# IU data charecteristics

In [ ]:
# Look at the dataframe

report_df_explode.head()

In [ ]:
# Adding sentance count feature for findings

report_df_explode["findings_sentance_count"] = (
    report_df_explode["findings"]
    .astype(str)
    .str.split(".")
    .apply(lambda x: 0 if x == None else len(x))
)
report_df_explode.head()

In [ ]:
# Remove duplicate reports so as only to have one instance per patient

report_df_dedup = report_df_explode.drop_duplicates(subset="id", keep="first")

In [ ]:
# Printing min,max and median of word_count

print(
    "Minimum number of word count for finding is:",
    np.min(report_df_dedup.findings_count.values),
)
print(
    "Maximum number of word count for finding is:",
    np.max(report_df_dedup.findings_count.values),
)
print(
    "Median number of word count for finding is:",
    np.median(report_df_dedup.findings_count.values),
)

In [ ]:
# Plotting top 50 frequent sentences of Findings feature

sentences = report_df_dedup["findings"].value_counts()[:50]
plt.figure(figsize=(20, 5))
sns.barplot(x=[el[:25] for el in sentences.index], y=sentences.values, alpha=0.8)
plt.ylabel("Number of Occurrences", fontsize=10)
plt.xticks(fontsize="large", rotation=90)
plt.title("Findings-Unique sentences")
plt.show()

In [ ]:
# Plot a word cloud of the top 500 words that appear in findings

wordcloud = WordCloud(
    max_words=500, background_color="black", colormap="Set3"
).generate(" ".join(report_df_dedup["findings"].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud, interpolation="Bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Printing min,max and median of word_count

print(
    "Minimum number of word count for Impression is:",
    np.min(report_df_dedup.impression_count.values),
)
print(
    "Maximum number of word count for Impression is:",
    np.max(report_df_dedup.impression_count.values),
)
print(
    "Median number of word count for Impression is:",
    np.median(report_df_dedup.impression_count.values),
)

In [ ]:
# Plotting top 50 frequent sentences of Impression feature

sentences = report_df_dedup["impression"].value_counts()[:50]
plt.figure(figsize=(20, 5))
sns.barplot(x=[el[:25] for el in sentences.index], y=sentences.values, alpha=0.8)
plt.ylabel("Number of Occurrences", fontsize=10)
plt.xticks(fontsize="large", rotation=90)
plt.title("Impression-Unique sentences")
plt.show()

In [ ]:
# Plot a word cloud of the top 500 words that appear in findings

wordcloud = WordCloud(
    max_words=500, background_color="black", colormap="Set3"
).generate(" ".join(report_df_dedup["impression"].astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud, interpolation="Bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Show the 10 most common mesh terms

mesh = report_df_dedup["mesh"]
p = Counter(" ".join(mesh).split()).most_common(10)

p

In [ ]:
# Show the unique mesh values

report_df_dedup["mesh"].unique().tolist()[:20]

In [ ]:
# Show the 10 most common impression terms

impression = report_df_dedup["impression"]
p = Counter(" ".join(impression).split()).most_common(10)

p

In [ ]:
# Show the 10 most common finding terms

finding = report_df_dedup["findings"]
p = Counter(" ".join(finding).split()).most_common(10)

p

In [ ]:
# Review images from IU dataset

In [ ]:
# Displaying sample 9 patient X-Ray
fig, axs = plt.subplots(5, 5, figsize=(9, 9), tight_layout=True)
for row, figure in zip(report_df_dedup[201:300].itertuples(), axs.flatten()):
    image = mpimg.imread(IMAGES_LOC + "/" + row.image_name.split(",")[0])
    figure.imshow(image)
plt.show()

In [ ]:
# Report figures

In [ ]:
plt.rcParams["figure.figsize"] = [10, 6]
# df3['image_count'].value_counts().plot.bar()
report_df_dedup.groupby("image_count").size().plot.bar()
plt.xlabel("Views per study")
plt.ylabel("Number of studies")
plt.savefig("./graph1.png")

In [ ]:
# end of script